# A Tool Developer's Guide to TensorFlow Model Files

Most users shouldn't need to care about the internal details of how TensorFlow stores data on disk, but you might if you're a tool deveoper. For example, you may want to analyze models, or convert back and forth between TensorFlow and other formats. This guide tries to explain some of the details of how you can work with the main files that hold model data, to make it easier to develop those kind of tools.

## Protocol Buffers

All of TensorFlow's file formats are based on [Protocol Buffers](https://developers.google.com/protocol-buffers), so to start it's worth getting familiar with how they work. The summary is that you define data structures in text files, and the `protobuf` tools generate classes in `C`, `Python`, and other languages that can load, save, and access the data in a friendly way. We often refer to `Protocol Buffers` as `protobufs`, and I'll use that convention in this guide.

## GraphDef

The foundation of computation in TensorFlow is the `Graph` object. This holds a network of nodes, each representing one operation, connected to each other as inputs and outputs. After you've created a `Graph` object, you can save it out by calling `as_graph_def()`, which returns a `GraphDef` object.

The `GraphDef` class is an object created by the `ProtoBuf` library from the definition in [tensorflow/core/framework/graph.proto](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/core/framework/graph.proto). The protobuf tools parse this text file, and generate the code to load, store, and manipulate graph definitions. If you see a standalone TensorFlow file representing a model, it's likely to contain a serialized version of one of these `GraphDef` objects saved out by the `protobuf` code.

This generated code is used to save and load the `GraphDef` files from disk. The code that actually loads the model looks like this:

```python
graph_def = graph_pb2.GraphDef()
```

This line creates an empty `GraphDef` object, the class that's been created from the textual definition in `graph.proto`. This is the object we're going to populate with the data from our file.

```python
with open(FLAGS.graph, 'rb') as f:
```

Here we get a file handle for the path we've passed into the script

```python
if FLAGS.input_binary:
    graph_def.ParseFromString(f.read())
else:
    text_format.Merge(f.read(), graph_def)
```

## Text or Binary?

There are actually two different formats that a `ProtoBuf` can be saved in. `TextFormat` is human-readable form, which makes it nice for debugging and editing, but can get large when there's numerical data like weights stored in it. You can see a small example of that in [graph_run_run2.pbtxt](https://github.com/tensorflow/tensorflow/blob/r0.11/tensorflow/tensorboard/components/tf-tensorboard/test/data/graph_run_run2.pbtxt).

Binary format files are a lot smaller than their text equivalents, even though they're not as readable for us. In this script, we ask the user to supply a flag indicating whether the input file is binary or text. You can find and example of a large binary file inside the [inception_v3 archive](https://storage.googleapis.com/download.tensorflow.org/models/inception_v3_2016_08_28_frozen.pb.tar.gz), as `inception_v3_2016_08_28_frozen.pb`.

The API itself can be a bit confusing - the binary call is actually `ParseFromString()`, whereas you use a utility function from the `text_format` module to load textual files.

## Nodes

Once you've loaded a file into the `graph_def` variable, you can now access the data inside it. For most practical purposes, the important section is the list of nodes stored in the node member. Here's the code that loops through those:

```python
for node in graph_def.node
```

Each node is a `NodeDef` object, defined in [tensorflow/core/framework/node_def.proto](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/core/framework/node_def.proto). These are fundamental building blocks of TensorFlow graphs, with each one defining a single operation along with its input connections. Here are the members of a `NodeDef`, and what they mean.

### `name`

Every node should have a unique identifier that's not used by any other nodes in the graph. If you don't specify one as you're building a graph using the Python API, one reflecting the name of operation, such as "MatMul", concatenated with a monotonically increasing number, such as "5", will be picked for you. The name is used when defining the connections between nodes, and when setting inputs and outputs for the whole graph when it's run.

### `op`

This defines what operation to run, for example "Add", "MatMul" or "Conv2D". When a graph is run, this `op` name is looked up in a registry to find an implementation. The registry is populated by calls to the `REGISTRER_OP()` macro, like those in [tensorflow/core/ops/nn_ops.cc](https://github.com/tensorflow/tensorflow/blob/master/tensorflow/core/ops/nn_ops.cc).

### `input`

A list of strings, each one of which is the name of another node, optionally followed by a colon and an output port number. For example, a node with two inputs might have a list like